## 작업형 3

#### 아래 E-Commerce Shipping Data의 train set을 참조하여 고객이 주문한 물품의 정시 도착 여부를 예측하시오. 
#### (ID와 예측치를 csv 파일로 저장하여 제출하시오)

In [20]:
import pandas as pd
test = pd.read_csv("X_test.csv")
X = pd.read_csv("X_train.csv")
y = pd.read_csv("y_train.csv")

X_num = X[['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']]
X_cat = X[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']]
X_cat=pd.get_dummies(X_cat)

test_num = test[['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']]
test_cat = test[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']]
test_cat = pd.get_dummies(test_cat)

X_cat, test_cat = X_cat.align(test_cat, join='inner', axis=1)

from sklearn.preprocessing import MinMaxScaler
minmax=MinMaxScaler()
minmax.fit(X_num)
X_scaled=minmax.transform(X_num)
test_scaled=minmax.transform(test_num)

X_final = pd.concat([pd.DataFrame(X_scaled), X_cat], axis=1)
test_final = pd.concat([pd.DataFrame(test_scaled), test_cat], axis=1)

y = y['Reached.on.Time_Y.N']

from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_final, y)

pred_test=model.predict_proba(test_final)
pred_test_prob = pd.DataFrame(pred_test[:, 1], columns = ['predict_prob'])
final_predict = pd.concat([test['ID'], pred_test_prob], axis=1)
# print(final_predict)
final_predict.to_csv("20211204.csv", index=False)

In [21]:
from sklearn.metrics import *
pred_train=model.predict_proba(X_final)

roc_auc_score(y, pred_train[:,1])

0.7260594803759932

In [24]:
import pandas as pd
test = pd.read_csv("X_test.csv")
X = pd.read_csv("X_train.csv")
y = pd.read_csv("y_train.csv")

X_str = X[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']]
X_num = X[['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']]
X_str = pd.get_dummies(X_str)

test_str = X[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']]
test_num = X[['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Discount_offered', 'Weight_in_gms']]
test_str = pd.get_dummies(test_str)

X_str, test_str = X_str.align(test_str, join='inner', axis=1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_num)
X_scaled = scaler.transform(X_num)
test_scaled = scaler.transform(test_num)

X_fin = pd.concat([X_str, pd.DataFrame(X_scaled)], axis=1)
test_fin = pd.concat([test_str, pd.DataFrame(test_scaled)], axis=1)

y = y['Reached.on.Time_Y.N']

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
model = RandomForestClassifier()
model.fit(X_fin, y)
pred_train = model.predict_proba(X_fin)
roc_auc_score(y, pred_train[:,1])


pred_test = model.predict_proba(test_fin)
pred_test_pos = pred_test[:,1]
#print(pred_train)
roc_auc_score(y, pred_test_pos)

a = pd.DataFrame(pred_test_pos, columns=['predict_prob'])
complete = pd.concat([test['ID'], a], axis=1)
# complete = pd.DataFrame({'ID':testID, 'predict_prob':pred_test_pos})
complete.to_csv("20211204.csv", index=False)